# IEA web scraping script
This workbook scrapes the IEA's API for country level (see list of excluded countries) data on electricity (GWh) use and heat use (TJ). Data are also scraped for large regions of the world (e.g., continents). 

In [ ]:
# import required packages
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from urllib import error
import requests
import json
import http.client as http
http.HTTPConnection._http_vsn = 10
http.HTTPConnection._http_vsn_str = 'HTTP/1.0'

In [ ]:
# countries excluded from IEA:
# excl_countries = ['AFGHANISTAN','ANDORRA','ANTIGUA-AND-BARBUDA','THE-BAHAMAS','BARBADOS','BELIZE','BHUTAN',
#                   'BURKINA FASO','BURUNDI','CABO-VERDE','CENTRAL-AFRICAN-REPUBLIC','CHAD','COMOROS',
#                   'DJIBOUTI','DOMINICA','EAST-TIMOR ','EQUATORIAL-GUINEA','ESWATINI','FIJI','THE-GAMBIA','GRENADA',
#                   'GUINEA','GUINEA-BISSAU','GUYANA','KIRIBATI','LAOS','LESOTHO','LIBERIA','LIECHTENSTEIN','MADAGASCAR',
#                   'MALAWI','MALDIVES','MALI','MALDIVES','MALI','MICRONESIA','MONACO','NAURU','PALAU','PAPUA-NEW-GUINEA',
#                   'RWANDA','SAINT-KITTS-AND-NEVIS','SAINT-LUCIA','SAINT-VINCENT-AND-THE-GRENADINES','SAMOA',
#                   'SAN-MARINO','SAO-TOME-AND-PRINCIPE','SEYCHELLES','SIERRA-LEONE','SOLOMON-ISLANDS','SOMALIA',
#                   'TAIWAN','TONGA','TRINIDAD-AND-TOBAGO','TUVALU','UGANDA','VANUATU','VATICAN-CITY']

#list of countries in the world included in the IEA dataset
countries = ['ALBANIA','ALGERIA','ANGOLA','ARGENTINA','ARMENIA','AUSTRALIA','AUSTRIA','AZERBAIJAN','BAHRAIN',
             'BANGLADESH','BELARUS','BELGIUM','BENIN','BOLIVIA','BOSNIA-AND-HERZEGOVINA','BOTSWANA','BRAZIL',
             'BRUNEI-DARUSSALAM','BULGARIA','CAMBODIA','CAMEROON','CANADA','CHILE','CHINA','CHINESE-TAIPEI',
             'COLOMBIA','CONGO','COSTA-RICA','COTE-DIVOIRE','CROATIA','CUBA','CURACAO','CYPRUS','CZECH-REPUBLIC',
             'DEMOCRATIC-PEOPLES-REPUBLIC-OF-KOREA','DEMOCRATIC-REPUBLIC-OF-THE-CONGO','DENMARK','DOMINICAN-REPUBLIC',
             'ECUADOR','EGYPT','EL-SALVADOR','ERITREA','ESTONIA','ETHIOPIA','FINLAND','FRANCE','GABON','GEORGIA',
             'GERMANY','GHANA','GIBRALTAR','GREECE','GUATEMALA','HAITI','HONDURAS','HUNGARY','HONG-KONG','ICELAND',
             'INDIA','INDONESIA','IRAN','IRAQ','IRELAND','ISRAEL','ITALY','JAMAICA','JAPAN','JORDAN','KAZAKHSTAN',
             'KENYA','KOREA','KOSOVO','KUWAIT','KYRGYZSTAN','LATVIA','LEBANON','LIBYA','LITHUANIA','LUXEMBOURG',
             'MALAYSIA','MALTA','MAURITIUS','MEXICO','MOLDOVA','MONGOLIA','MONTENEGRO','MOROCCO','MOZAMBIQUE',
             'MYANMAR','NAMIBIA','NEPAL','NETHERLANDS','NEW-ZEALAND','NICARAGUA','NIGER','NIGERIA','NORTH-MACEDONIA',
             'NORWAY','OMAN','PAKISTAN','PANAMA','PARAGUAY','PERU','PHILIPPINES','POLAND','PORTUGAL','QATAR','ROMANIA',
             'RUSSIAN-FEDERATION','SAUDI-ARABIA','SENEGAL','SERBIA','SINGAPORE','SLOVAK-REPUBLIC','SLOVENIA',
             'SOUTH-AFRICA','SPAIN','SRI-LANKA','SUDAN','SOUTH-SUDAN','SURINAME','SWEDEN','SWITZERLAND','SYRIA',
             'TANZANIA','TAJIKISTAN','THAILAND','TOGO','TUNISIA','TURKEY','TURKMENISTAN','UKRAINE','UNITED-ARAB-EMIRATES',
             'UNITED-KINGDOM','UNITED-STATES','URUGUAY','UZBEKISTAN','VENEZUELA','VIETNAM','YEMEN','ZAMBIA',
             'ZIMBABWE','AFRICA','ASIA','LATIN-AMERICA','MASEAN','MIDDLE-EAST','NON-OECD','OECD','OECDAM','OECDAO',
             'OCEDEUR','WEOAFRICA','WEOASIAPAC','WEOCSAM','WEOEURASIA','WEOMIDEAST','WEONAM','WORLD']

#list of years to scrape data for 
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018']

#pull raw data, separate into electricity and heat, and save as csv
for year in years:
    df = []
    for country in countries: 
        url = 'https://api.iea.org/stats/?year='+year+'&countries='+country+'&series=ELECTRICITYANDHEAT'
        try:
            requests.get(url)
            r = urlopen(url).read()
            data = pd.read_json(r)
            df.append(data)        
        # skip any errors occurring from nonexistent urls
        except error.HTTPError as e:
            print('HTTP error.', 'Skipping url:', url)
            print('Error code: ', e.code)
            continue
    df = pd.concat(df) #append dataframes
    df = df.drop(columns=['flow','flowOrder','product']) #drop unnecessary columns
    #make new dataframes, separated by electricity and heat
    electricity = df[df.productOrder == 1]
    heat = df[df.productOrder == 2]
    #drop redundant columns in new dataframes and re-index
    electricity = electricity.drop(columns=['productLabel','productOrder','units', 'year','country']
                              ).reset_index(drop=True) #GWh
    heat = heat.drop(columns=['productLabel','productOrder','units', 'year','country']
                              ).reset_index(drop=True) #TJ
    #pivot raw data
    electricity = electricity.pivot_table(index='short',columns='flowLabel',values='value')
    heat = heat.pivot_table(index='short',columns='flowLabel',values='value')
    #save as csv
    electricity.to_csv('IEA-electricity-GWh-'+year+'.csv')
    heat.to_csv('IEA-heat-TJ-'+year+'.csv')

In [ ]:
#separate electricity production (generation) and calculate electricity mix (as % of total gen), save as csv's
d = {}
for year in years:
    d[year] = pd.read_csv('IEA-electricity-GWh-'+year+'.csv', index_col='short').drop(['Agriculture / forestry',
                                                                                       'Commercial and public services', 
                                                                                       'Domestic supply',
                                                                                       'Energy industry own use', 'Exports',
                                                                                       'Final consumption', 'Fishing', 
                                                                                       'Heat plants', 'Imports', 'Industry', 
                                                                                       'Losses', 'Municipal Waste', 
                                                                                       'Other non-specified', 
                                                                                       'Residential','Statistical differences', 
                                                                                       'Transformation', 'Transport', 
                                                                                       'Waste (renewable)'], axis=1)
    d[year] = d[year][[c for c in electricity_2010 if c not in ['Total production']] 
       + ['Total production']]
    d[year].to_csv('IEA-electricity-mix-'+year+'-GWh.csv')
    
    
p = {}
for year in years:
    p[year] = d[year]
    nrows = len(p[year])
    for n in range(0,nrows):
        value = d[year].iloc[n,13]
        p[year].iloc[n,:] = (d[year].iloc[n,:]/value)*100
    p[year].to_csv('IEA-electricity-mix-'+year+'-percent.csv')

In [ ]:
#separate fuel energy for heat, calculate fuel mix (as % of total fuel energy), save as csv's
df = {}
for year in years:
    df[year] = pd.read_csv('IEA-heat-TJ-'+year+'.csv', index_col='short').drop(['Agriculture / forestry',
                                                                               'Commercial and public services', 
                                                                               'Domestic supply','Electricity plants', 
                                                                               'Energy industry own use', 'Exports',
                                                                               'Final consumption', 'Fishing',
                                                                               'Heat plants','Imports', 
                                                                               'Industry', 'Losses', 'Municipal Waste',
                                                                               'Other non-specified','Residential',
                                                                               'Statistical differences', 
                                                                               'Transformation','Waste (renewable)'], 
                                                                               axis=1)
    df[year] = df[year][[c for c in df[year] if c not in ['Total production']] 
       + ['Total production']]
    df[year].to_csv('IEA-heat-mix-'+year+'-TJ.csv')
    
    
pf = {}
for year in years:
    pf[year] = df[year]
    nrows = len(pf[year])
    for n in range(0,nrows):
        value = df[year].iloc[n,9]
        pf[year].iloc[n,:] = (df[year].iloc[n,:]/value)*100
    pf[year].to_csv('IEA-heat-mix-'+year+'-percent.csv')